In [12]:
import re
import json
from models.chat_model import ChatModel
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
def member_func(name: str):
    members = {
        "James": "James is a member of A team",
        "Jim": "Jim is a member of B team",
        "Jimmy": "Jimmy is a member of C team",
        "Jimothy": "Jimothy is a member D the team",
    }
    return f"{members[name]}"


member_tool = {
    "name": "Member",
    "func": member_func
}

In [19]:
STRUCTURED_TOOL_SYSTEM = 'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nMember: useful for when you query someone belongs to which team, args: {{\'name\': {{\'title\': \'Name\', \'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or Member\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}\n```\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation:.\nThought:'

USER_MESSAGE_TEMPLATE = "{input}\n\n{agent_scratchpad}"

In [7]:
messages1 = [{'role': 'system', 'content': 'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nMember: useful for when you query someone belongs to which team, args: {{\'name\': {{\'title\': \'Name\', \'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or Member\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}\n```\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation:.\nThought:'}, {
    'role': 'user', 'content': 'James belongs to which team?\n\n'}]

In [8]:
step1 = model(messages=messages1, stop=stop)
print(step1)

I can use the Member tool to find out which team James belongs to. Let me do that for you.
Action:
```
{
  "action": "Member",
  "action_input": {
    "name": "James"
  }
}
```


In [ ]:
class PromptTemplate(object):
    def __init__(self, input_variables, messages):
        self.input_variables = input_variables
        self.messages = messages

    def format(self, **kwargs):
        messages = []
        for message in messages:
            text = message['template'].format(**kwargs)
            messages.append({
                "role": message['role'],
                "content": text
            })
        return messages

In [ ]:
class Agent(object):
    def __init__(self, tools, stop=['Observation:'], max_iterations=15, **kwargs):
        self.model = ChatModel()
        self._stop = stop
        self.name_to_tool_map = self._re_tools(tools)

        messages = [{
            "role": "system",
            "template": STRUCTURED_TOOL_SYSTEM,
        }, {
            "role": "user",
            "template": USER_MESSAGE_TEMPLATE,
        }]
        self.prompt = PromptTemplate(['input', 'agent_scratchpad'], messages)

        self.system_prompt = STRUCTURED_TOOL_SYSTEM
        self.user_prompt_template = USER_MESSAGE_TEMPLATE
        self.max_iterations = max_iterations
        self.observation_prefix = kwargs.get(
            'observation_prefix', "Observation: ")
        self.llm_prefix = kwargs.get('llm_prefix', "Thought:")

    def _re_tools(self, tools):
        name_to_tool_map = {}
        for tool in tools:
            name_to_tool_map[tool["name"]] = {
                "run": tool["func"]
            }
        return name_to_tool_map

    def _construct_scratchpad(self, intermediate_steps):
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\n{self.observation_prefix}{observation}\n{self.llm_prefix}"
        return (
            f"This was your previous work "
            f"(but I haven't seen any of it! I only see what "
            f"you return as final answer):\n{thoughts}"
        )

    def call(self, input):
        intermediate_steps = []
        iterations = 0
        while iterations <= self.max_iterations:
            next_step_output = self._take_next_setp(input, intermediate_steps)
            if (next_step_output['type'] is 'finished'):
                return next_step_output

            intermediate_steps.extend(next_step_output)
            iterations += 1

    def _output_parse(self, text):
        try:
            action_match = re.search(r"```(.*?)```?", text, re.DOTALL)
            if action_match is not None:
                response = json.loads(
                    action_match.group(1).strip(), strict=False)
                if isinstance(response, list):
                    # gpt turbo frequently ignores the directive to emit a single action
                    response = response[0]
                if response["action"] == "Final Answer":
                    return {
                        "type": "finished",
                        "output": response["action_input"],
                        "text": text
                    }
                else:
                    return {
                        "type": "step",
                        "tool": response["action"],
                        "tool_input": response.get("action_input", {}),
                        "log": text
                    }
            else:
                return {
                    "type": "finished",
                    "output": text,
                    "text": text
                }
        except Exception as e:
            raise Exception(f"Could not parse LLM output: {text}") from e

    def get_full_inputs(self, intermediate_steps, **kwargs):
        """Create the full inputs for the llm from intermediate steps."""
        thoughts = self._construct_scratchpad(intermediate_steps)
        new_inputs = {"agent_scratchpad": thoughts, "stop": self._stop}
        full_inputs = {**kwargs, **new_inputs}
        return full_inputs

    def prep_prompts(self, input_list):
        """Prepare prompts from inputs."""
        stop = None
        if "stop" in input_list[0]:
            stop = input_list[0]["stop"]
        prompts = []
        for inputs in input_list:
            selected_inputs = {k: inputs[k] for k in self.prompt.input_variables}
            prompt = self.prompt.format(**selected_inputs)
            prompts.append(prompt)

        return prompts, stop

    def _take_next_setp(self, intermediate_steps, **kwargs):
        full_inputs = self.get_full_inputs(intermediate_steps, **kwargs)

        prompts, stop = self.prep_prompts([full_inputs])

        response = self.model(messages=prompts, stop=stop)
        output = self._output_parse(response)
        
        if (output['type'] is 'finished'):
            return output
        actions = []
        if (output['tool'] is not None):
            actions = [output]
        else:
            actions = output
        result = []
        for action in actions:
            if action.tool in self.name_to_tool_map:
                tool = self.name_to_tool_map[action.tool]
                observation = tool['run'](action['tool_input'])
                result.append((action, observation))

In [9]:
messages2 = [{'role': 'system', 'content': 'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nMember: useful for when you query someone belongs to which team, args: {{\'name\': {{\'title\': \'Name\', \'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or Member\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}\n```\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation:.\nThought:'}, {
    'role': 'user', 'content': 'James belongs to which team?\n\nThis was your previous work (but I haven\'t seen any of it! I only see what you return as final answer):\nI can use the Member tool to find out which team James belongs to.\nAction:\n```\n{\n  "action": "Member",\n  "action_input": {\n    "name": "James"\n  }\n}\n```\nObservation: James is a member of A team\nThought:'}]
step2 = model(messages=messages2, stop=stop)
print(step2)

James belongs to team A.
Action:
```json
{
  "action": "Final Answer",
  "action_input": "James belongs to team A."
}
```


In [10]:
messages3 = [{'role': 'user', 'content': 'Instructions:\n--------------\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}}}\n```\n--------------\nCompletion:\n--------------\nJames belongs to team A.\nAction:\n```json\n{\n  "action": "Final Answer",\n  "action_input": "James belongs to team A."\n}\n```\n--------------\n\nAbove, the Completion did not satisfy the constraints given in the Instructions.\nError:\n--------------\nOutputParserException(\'Could not parse LLM output: James belongs to team A.\\nAction:\\n```json\\n{\\n  "action": "Final Answer",\\n  "action_input": "James belongs to team A."\\n}\\n```\')\n--------------\n\nPlease try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions:'}]
step3 = model(messages=messages3, stop=stop)
print(step3)

James belongs to team A.
Thought: This is the final answer to the question.
Action:
```
{
  "action": "Final Answer",
  "action_input": "James belongs to team A."
}
```
